# RAG(Retrieval-Augmented Generation)

검색 증강 생성(RAG)은 근거 데이터를 제공하는 정보 검색 시스템을 추가하여 ChatGPT와 같은 대규모 언어 모델(LLM)의 기능을 보강하는 아키텍처입니다.</br>
정보 검색 시스템을 추가하면 LLM이 응답시 사용하는 근거 데이터를 제어할 수 있습니다.</br>
엔터프라이즈 솔루션의 경우, RAG 아키텍처는 벡터화된 문서와 이미지, 그리고 해당 콘텐츠에 대한 임베딩(embedding) 모델이 있는 경우 기타 데이터 소스로부터 수집 된 엔터프라이즈 컨텐츠로 생성형 AI를 제한할 수 있다는 것을 의미합니다.

더 자세한 내용은 [Azure AI 검색을 사용하여 RAG 솔루션을 빌드하는 방법](https://learn.microsoft.com/ko-kr/azure/search/tutorial-rag-build-solution) 에서 확인 할 수 있습니다.